<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/HyperParameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Machine-Learning.git cloned-repo
%cd cloned-repo
!ls

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [0]:
bc_data = pd.read_csv('breast_cancer_data.csv')
#view top 10 rows
bc_data.head(10)


In [0]:
bc_data.columns
bc_data.drop(index=1,columns='Unnamed: 32')

In [0]:
#convert diagnosis to 0= benign, 1= maglignant
#bc_data.iloc[:,1]
def convert_diagnosis(x):
  if(x=='M'):
    return '1'
  if(x=='B'):
    return '0'

bc_data['c_diagnosis'] = bc_data['diagnosis'].apply(convert_diagnosis)

In [0]:
bc_data.columns

In [0]:
bc_data.pop('diagnosis')
bc_data.pop('Unnamed: 32')
bc_data.tail(20)

In [0]:
#Split data into features and labels
X = bc_data.drop(['c_diagnosis'],axis=1)
y = bc_data['c_diagnosis']

In [0]:
#perform training and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
train_stats = X_train.describe()
#train_stats.pop("c_diagnosis")
train_stats = train_stats.transpose()
train_stats

In [0]:
test_stats = X_test.describe()
test_stats = test_stats.transpose()
test_stats

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(X_train)
normed_test_data = norm(X_test)
print("done")

In [0]:
#Logistic regression
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_train,y_train)
y_pred = clf.predict(X_test)

#Logistic Regression Classifier Confusion matrix
from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

In [0]:
inputs = len(X_train.keys())
print("number of inputs to the model = " + str(inputs))

def build_model():
  model = keras.Sequential([
    layers.Dense(16, activation=tf.nn.relu,input_shape=([len(X_train.keys())]),),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model
  print("done")

In [0]:
model = build_model()
print("done")

In [0]:
EPOCHS = 1

history = model.fit(
  normed_train_data, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1
  )

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)